In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import njit
from scipy import sparse

from graph_ml.datasets.polbooks import PolBooks
from graph_ml.utils import utils
# graph = nx.erdos_renyi_graph(10, 0.2, seed=92)
# A = nx.adjacency_matrix(graph)
# nx.draw(graph)
# np.array(A.todense())

In [2]:
A, group_membership = PolBooks().adj, PolBooks().y
A.toarray(), group_membership

Extracting /tmp/polbooks.zip
Using existing file polbooks.zip
Extracting /tmp/polbooks.zip


(array([[0, 1, 1, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 1, 0]]),
 array([0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0]))

In [3]:
window_length = 10
dcsbm = True
group_membership = np.unique(group_membership, return_inverse=True)[1]
group_membership

array([0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0])

In [4]:
n_nodes = A.shape[0]
node2group = utils.to_member_matrix(group_membership)

In [5]:
indeg = np.array(A.sum(axis=1)).reshape(-1)
Lambda = (node2group.T @ A @ node2group).toarray()
Din = np.array(np.sum(Lambda, axis=0)).reshape(-1)
Nin = np.array(node2group.sum(axis=0)).reshape(-1)
Psbm = np.einsum('ij,i->ij', Lambda, 1 / np.maximum(1, np.array(np.sum(Lambda, axis=1))))
Psbm_pow = utils.matrix_sum_power(Psbm, window_length) / window_length

block2node = (
    sparse.diags(1 / np.maximum(1, Din))
    @ sparse.csr_matrix(
        node2group.T
    )
    @ sparse.diags(indeg)
)




In [6]:
block2block = sparse.csr_matrix(
    Psbm_pow
)
block2block.data = utils.csr_row_cumsum(block2block.indptr, block2block.data)

block2node.data = utils.csr_row_cumsum(block2node.indptr, block2node.data)

In [9]:
block2node.toarray().shape

(3, 105)

In [10]:
block2block.toarray()

array([[0.10477346, 0.60810079, 1.        ],
       [0.08979549, 0.8052493 , 1.        ],
       [0.07837984, 0.29670563, 1.        ]])